# SpotOptim Demonstrations

This notebook demonstrates the usage of the SpotOptim optimizer for various optimization problems.

## Example 1: 2-Dimensional Rosenbrock Function

This example shows basic optimization on the classic 2D Rosenbrock function.

In [ ]:
import numpy as np
from spotoptim.SpotOptim import SpotOptim

# Define Rosenbrock function
def rosenbrock(X):
    """Rosenbrock function for optimization."""
    X = np.atleast_2d(X)
    x = X[:, 0]
    y = X[:, 1]
    return (1 - x)**2 + 100 * (y - x**2)**2

# Set up bounds for 2D problem
bounds = [(-2, 2), (-2, 2)]

# Create optimizer
optimizer = SpotOptim(
    fun=rosenbrock,
    bounds=bounds,
    max_iter=50,
    n_initial=5,
    acquisition='ei',
    seed=42,
    verbose=False
)

# Run optimization
result = optimizer.optimize()

# Print results
print("\n" + "="*50)
print("Optimization Results")
print("="*50)
print(f"Best point found: {result.x}")
print(f"Best function value: {result.fun:.6f}")
print(f"Number of function evaluations: {result.nfev}")
print(f"Number of iterations: {result.nit}")
print(f"Success: {result.success}")
print(f"Message: {result.message}")
print("\nTrue optimum: [1, 1] with f(x) = 0")

## Example 2: 6-Dimensional Rosenbrock Function

This example demonstrates optimization of the 6-dimensional Rosenbrock function with a budget of 100 total function evaluations.

In [2]:
import numpy as np
from spotoptim.SpotOptim import SpotOptim

# Define 6-dimensional Rosenbrock function
def rosenbrock_6d(X):
    """
    6-dimensional Rosenbrock function for optimization.
    
    The Rosenbrock function is defined as:
    f(x) = sum_{i=1}^{n-1} [100*(x_{i+1} - x_i^2)^2 + (1 - x_i)^2]
    
    Global minimum: f(1, 1, 1, 1, 1, 1) = 0
    """
    X = np.atleast_2d(X)
    n_samples, n_dim = X.shape
    
    result = np.zeros(n_samples)
    for i in range(n_samples):
        x = X[i]
        total = 0
        for j in range(n_dim - 1):
            total += 100 * (x[j+1] - x[j]**2)**2 + (1 - x[j])**2
        result[i] = total
    
    return result

# Set up bounds for 6D problem
# Typical search domain for Rosenbrock is [-5, 10] for each dimension
bounds_6d = [(-2, 2)] * 6

# Budget: 100 total function evaluations
# Split into initial design and optimization iterations
n_total = 100
n_initial = 6  # Initial Latin Hypercube Design points
max_iter = n_total - n_initial   # Optimization iterations: 6 + 94 = 100 total evaluations

print("="*60)
print("6D Rosenbrock Function Optimization")
print("="*60)
print(f"Problem dimension: 6")
print(f"Search bounds: [-2, 2] for each dimension")
print(f"Total budget: {n_initial + max_iter} function evaluations")
print(f"  - Initial design: {n_initial} points")
print(f"  - Optimization iterations: {max_iter}")
print(f"Global optimum: x* = [1, 1, 1, 1, 1, 1], f(x*) = 0")
print("="*60)

# Create optimizer with Expected Improvement acquisition
optimizer_6d = SpotOptim(
    fun=rosenbrock_6d,
    bounds=bounds_6d,
    max_iter=max_iter,
    n_initial=n_initial,
    acquisition='y',  # Expected Improvement
    seed=42,
    verbose=True
)

# Run optimization
print("\nStarting optimization...\n")
result_6d = optimizer_6d.optimize()

# Print final results
print("\n" + "="*60)
print("Final Optimization Results")
print("="*60)
print(f"Best point found: {result_6d.x}")
print(f"Best function value: {result_6d.fun:.6e}")
print(f"Number of function evaluations: {result_6d.nfev}")
print(f"Number of iterations: {result_6d.nit}")
print(f"Success: {result_6d.success}")

# Calculate distance from true optimum
true_optimum = np.ones(6)
distance_to_optimum = np.linalg.norm(result_6d.x - true_optimum)
print(f"\nDistance to true optimum [1,1,1,1,1,1]: {distance_to_optimum:.6f}")

# Show improvement over initial design
initial_best = np.min(result_6d.y[:n_initial])
final_best = result_6d.fun
improvement = initial_best - final_best
improvement_pct = (improvement / initial_best) * 100

print(f"\nImprovement analysis:")
print(f"  Best initial value: {initial_best:.6e}")
print(f"  Final best value: {final_best:.6e}")
print(f"  Absolute improvement: {improvement:.6e}")
print(f"  Relative improvement: {improvement_pct:.2f}%")
print("="*60)

6D Rosenbrock Function Optimization
Problem dimension: 6
Search bounds: [-2, 2] for each dimension
Total budget: 100 function evaluations
  - Initial design: 6 points
  - Optimization iterations: 94
Global optimum: x* = [1, 1, 1, 1, 1, 1], f(x*) = 0

Starting optimization...

Initial best: f(x) = 881.567459
Iteration 1: f(x) = 2103.440995
Iteration 2: New best f(x) = 881.329928
Iteration 3: f(x) = 969.604447
Iteration 4: New best f(x) = 596.767694
Iteration 5: New best f(x) = 381.514887
Iteration 6: New best f(x) = 346.681434
Iteration 7: f(x) = 402.306119
Iteration 5: New best f(x) = 381.514887
Iteration 6: New best f(x) = 346.681434
Iteration 7: f(x) = 402.306119
Iteration 8: New best f(x) = 164.872575
Iteration 9: New best f(x) = 97.516991
Iteration 8: New best f(x) = 164.872575
Iteration 9: New best f(x) = 97.516991
Iteration 10: New best f(x) = 79.141491
Iteration 11: New best f(x) = 76.288180
Iteration 10: New best f(x) = 79.141491
Iteration 11: New best f(x) = 76.288180
Iteratio